# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 66 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.15516


extracting tarball to tmp_2303.15516...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.15525


extracting tarball to tmp_2303.15525...

...

✔ → 84330:\section{Discussion} \label{sect:discussion}
  ↳ 98370:\section{Summary and Conclusions} \label{sect:summary and conclusions}
✔ → 98370:\section{Summary and Conclusions} \label{sect:summary and conclusions}
  ↳ 104237:\begin{appendix}
✔ → 104237:\begin{appendix}
  ↳ 104254:\section{Rayleigh adjustment for a nonuniform radial grid} \label{appendix:rayleigh adjustment}


✘ → 104254:\section{Rayleigh adjustment for a nonuniform radial grid} \label{appendix:rayleigh adjustment}
  ↳ 113054:\section{Dependence of the steady-state MRI-driven accretion on the power-law dust size distribution} \label{appendix:Follow-up on the effect of a dust distribution on the steady-state MRI-driven accretion regime}


✔ → 113054:\section{Dependence of the steady-state MRI-driven accretion on the power-law dust size distribution} \label{appendix:Follow-up on the effect of a dust distribution on the steady-state MRI-driven accretion regime}
  ↳ 120359:\section{Temporal evolution of the dust surface density distribution for Models V and VI} \label{appendix:temporal evolution of the dust surface density distribution for Models V and VI}
✔ → 120359:\section{Temporal evolution of the dust surface density distribution for Models V and VI} \label{appendix:temporal evolution of the dust surface density distribution for Models V and VI}
  ↳ 121988:end


Found 107 bibliographic references in tmp_2303.15675/aa_original_tdelage.bbl.
Retrieving document from  https://arxiv.org/e-print/2303.16137


extracting tarball to tmp_2303.16137... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.15525-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.15525) | **Euclid preparation. XXXII. A UV-NIR spectral atlas of compact planetary  nebulae for wavelength calibration**  |
|| E. Collaboration, et al. -- incl., <mark>K. Paterson</mark>, <mark>M. Schirmer</mark> |
|*Appeared on*| *2023-03-29*|
|*Comments*| *Accepted in A&A*|
|**Abstract**| The Euclid mission will conduct an extragalactic survey over 15000 deg$^2$ of the extragalactic sky. The spectroscopic channel of the Near-Infrared Spectrometer and Photometer (NISP) has a resolution of $R\sim450$ for its blue and red grisms that collectively cover the $0.93$--$1.89 $\micron;range. NISP will obtain spectroscopic redshifts for $3\times10^7$ galaxies for the experiments on galaxy clustering, baryonic acoustic oscillations, and redshift space distortion. The wavelength calibration must be accurate within $5$\AA to avoid systematics in the redshifts and downstream cosmological parameters. The NISP pre-flight dispersion laws for the grisms were obtained on the ground using a Fabry-Perot etalon. Launch vibrations, zero gravity conditions, and thermal stabilisation may alter these dispersion laws, requiring an in-flight recalibration. To this end, we use the emission lines in the spectra of compact planetary nebulae (PNe), which were selected from a PN data base. To ensure completeness of the PN sample, we developed a novel technique to identify compact and strong line emitters in Gaia spectroscopic data using the Gaia spectra shape coefficients. We obtained VLT/X-SHOOTER spectra from $0.3$ to $2.5$ \micron;for 19 PNe in excellent seeing conditions and a wide slit, mimicking Euclid's slitless spectroscopy mode but with 10 times higher spectral resolution. Additional observations of one northern PN were obtained in the $0.80$--$1.90$ \micron range with the GMOS and GNIRS instruments at the Gemini North observatory. The collected spectra were combined into an atlas of heliocentric vacuum wavelengths with a joint statistical and systematic accuracy of 0.1 \AA in the optical and 0.3 \AA in the near-infrared. The wavelength atlas and the related 1D and 2D spectra are made publicly available. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.15675-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.15675) | **The impact of dust evolution on the dead zone outer edge in magnetized  protoplanetary disks**  |
|| T. N. Delage, et al. -- incl., <mark>M. Gárate</mark>, <mark>M. Flock</mark> |
|*Appeared on*| *2023-03-29*|
|*Comments*| *23 pages, 13 figures, Accepted for publication in A&A*|
|**Abstract**| [Abridged] Aims. We provide an important step toward a better understanding of the magnetorotational instability (MRI)-dust coevolution in protoplanetary disks by presenting a proof of concept that dust evolution ultimately plays a crucial role in the MRI activity. Methods. First, we study how a fixed power-law dust size distribution with varying parameters impacts the MRI activity, especially the steady-state MRI-driven accretion, by employing and improving our previous 1+1D MRI-driven turbulence model. Second, we relax the steady-state accretion assumption in this disk accretion model, and partially couple it to a dust evolution model in order to investigate how the evolution of dust (dynamics and grain growth processes combined) and MRI-driven accretion are intertwined on million-year timescales. Results. Dust coagulation and settling lead to a higher gas ionization degree in the protoplanetary disk, resulting in stronger MRI-driven turbulence as well as a more compact dead zone. On the other hand, fragmentation has an opposite effect because it replenishes the disk in small dust particles. Since the dust content of the disk decreases over million years of evolution due to radial drift, the MRI-driven turbulence overall becomes stronger and the dead zone more compact until the disk dust-gas mixture eventually behaves as a grain-free plasma. Furthermore, our results show that dust evolution alone does not lead to a complete reactivation of the dead zone. Conclusions. The MRI activity evolution (hence the temporal evolution of the MRI-induced $\alpha$-parameter) is controlled by dust evolution and occurs on a timescale of local dust growth, as long as there is enough dust particles in the disk to dominate the recombination process for the ionization chemistry. Once it is no longer the case, it is expected to be controlled by gas evolution and occurs on a viscous evolution timescale. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.15516-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.15516) | **A systematic survey of millimetre-wavelength flaring variability of  Young Stellar Objects in the Orion Nebula Cluster**  |
|| J. Vargas-González, et al. -- incl., <mark>M. Güdel</mark> |
|*Appeared on*| *2023-03-29*|
|*Comments*| *15 pages, 8 figures, accepted for publication in MNRAS*|
|**Abstract**| High-energy processes are ubiquitous even in the earliest stages of protostellar evolution. Motivated by the results of our systematic search for intense centimeter radio flares in Young Stellar Objects (YSOs) and by rare findings of strong millimeter-wavelength variability, we have conducted a systematic search for such variability in the Orion Nebula Cluster (ONC) using Atacama Large Millimeter/submillimeter Array (ALMA). Rapid variability on timescales of minutes to hours in the (centimeter)millimeter-wavelength range indicates (gyro)synchrotron radiation. Additionally, mass accretion will also affect the millimeter-wavelength luminosity but typically on longer timescales. Beyond studies of individual YSOs, our characterization of strong millimeter-wavelength variability with ALMA in the ONC sets first systematic constraints on the occurrence of such variability in a large number of YSOs ($\sim$130). We report the discovery of an order of magnitude millimeter-flare within just a few minutes from a known YSO previously reported as a radio flaring source at cm-wavelengths (the "ORBS'' source). We also present an assessment of the systematic variability effects caused by the use of time-sliced imaging of a complex region. These are mostly due to the impact of a changing synthesized beam throughout the observations. We use simulated ALMA observations to reproduce and quantify these effects and set a lower limit for the variability that can be studied using our method in a complex region such as the ONC. Our results demonstrate that the utility of time domain analysis of YSOs extends into the millimeter-wavelength range, potentially interfering with the conversion of observed fluxes into dust masses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.15618-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.15618) | **Revisiting Emission-Line Measurement Methods for Narrow-Line Active  Galactic Nuclei**  |
|| V. C. Khatu, et al. -- incl., <mark>S.-S. Li</mark> |
|*Appeared on*| *2023-03-29*|
|*Comments*| *14 pages, 9 figures, accepted for publication in PASP*|
|**Abstract**| Measuring broad emission-line widths in active galactic nuclei (AGN) is not straightforward owing to the complex nature of flux variability in these systems. Line-width measurements become especially challenging when signal-to-noise is low, profiles are narrower, or spectral resolution is low. We conducted an extensive correlation analysis between emission-line measurements from the optical spectra of Markarian 142 (Mrk 142; a narrow-line Seyfert galaxy) taken with the Gemini North Telescope (Gemini) at a spectral resolution of 185.6+\-10.2 km/s and the Lijiang Telescope (LJT) at 695.2+\-3.9 km/s to investigate the disparities in the measured broad-line widths from both telescope data. Mrk~142 posed a challenge due to its narrow broad-line profiles, which were severely affected by instrumental broadening in the lower-resolution LJT spectra. We discovered that allowing the narrow-line flux of permitted lines having broad and narrow components to vary during spectral fitting caused a leak in the narrow-line flux to the broad component, resulting in broader broad-line widths in the LJT spectra. Fixing the narrow-line flux ratios constrained the flux leak and yielded the Hydrogen-beta broad-line widths from LJT spectra $\sim$54\% closer to the Gemini Hydrogen-beta widths than with flexible narrow-line ratios. The availability of spectra at different resolutions presented this unique opportunity to inspect how spectral resolution affected emission-line profiles in our data and adopt a unique method to accurately measure broad-line widths. Reconsidering line-measurement methods while studying diverse AGN populations is critical for the success of future reverberation-mapping studies. Based on the technique used in this work, we offer recommendations for measuring line widths in narrow-line AGN. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16137-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16137) | **XRBcats: Galactic High Mass X-ray Binary Catalogue**  |
|| <mark>M. Neumann</mark>, A. Avakyan, V. Doroshenko, A. Santangelo |
|*Appeared on*| *2023-03-29*|
|*Comments*| *10 pages, 2 figures, submitted to A&A. For auxillary files, see this http URL*|
|**Abstract**| We present a new catalogue of the high-mass X-ray binaries (HMXBs) in the Galaxy improving upon the most recent such catalogue. We include new HMXBs discovered since aforementioned publication and revise the classification for several objects previously considered HMXBs or candidates. The catalogue includes both basic information such as source names, coordinates, types, and more detailed data such as distance and X-ray luminosity estimates, binary system parameters and other characteristic properties of 169 HMXBs, together with appropriate references to the literature. Finding charts in several bands from infra-red to hard X-rays are also included for each object. The aim of this catalogue is to provide the reader a list of all currently known Galactic HMXBs with some basic information on both compact objects and non-degenerate counterpart properties (where available). We also include objects tentatively classified as HXMBs in the literature and give a brief motivation for the classifcation in each relevant case. The catalogue is compiled based on a search of known HMXBs and candidates in all commonly available databases and literature published before 31 October 2022. Relevant properties in the optical and other bands were collected for all objects either from the literature or using the data provided by large-scale surveys. In the later case, the counterparts in each individual survey were found by cross-correlating positions of identified HMXBs with relevant databases. An up-to date catalogue of Galactic HMXBs is presented to facilitate research in this area. An attempt was made to collect a larger set of relevant HMXB properties in a more uniform way compared to previously published works. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.15525.md
    + _build/html/tmp_2303.15525/./PNmorph_linear.png
    + _build/html/tmp_2303.15525/./PN_gaia.png
    + _build/html/tmp_2303.15525/./fp_pn.png
exported in  _build/html/2303.15675.md
    + _build/html/tmp_2303.15675/./impact_full_dust_evolution/Steady_state_vfrag1e2/Steady_MRI_v_frag1e2_dustpy_optimal_B_field.png
    + _build/html/tmp_2303.15675/./impact_dust_distribution_only/Steady-state_Vary_a_distMAX.png
    + _build/html/tmp_2303.15675/./impact_full_dust_evolution/LBP_vfrag1e2/LBP_MRI_v_frag1e2_dustpy_optimal_B_field.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ preparation. XXXII. A UV–NIR spectral atlas of compact planetary nebulae for wavelength calibration

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.15525-b31b1b.svg)](https://arxiv.org/abs/2303.15525)<mark>Appeared on: 2023-03-29</mark> -  _Accepted in A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Paterson</mark>, <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** The Euclid mission will conduct an extragalactic survey over 15 000 deg $^2$ of the extragalactic sky. The spectroscopic channel of the Near-Infrared Spectrometer and Photometer (NISP)has a resolution of $R\sim450$ for its blue and red grisms that collectively cover the $0.93$ -- $1.89 $ $\micron$ \; range. NISP will obtain spectroscopic redshifts for $3\times10^7$ galaxies for the experiments on galaxy clustering, baryonic acoustic oscillations, and redshift space distortion. The wavelength calibration must be accurate within $5 $ Å to avoid systematics in the redshifts and downstream cosmological parameters. The NISP pre-flight dispersion laws for the grisms were obtained on the ground using a Fabry-Perot etalon. Launch vibrations, zero gravity conditions, and thermal stabilisation may alter these dispersion laws, requiring an in-flight recalibration. To this end, we use the emission lines in the spectra of compact planetary nebulae (PNe), which were selected from a PN data base. To ensure completeness of the PN sample, we developed a novel technique to identify compact and strong line emitters in Gaia spectroscopic data using the Gaia spectra shape coefficients. We obtained VLT/X-SHOOTER spectra from $0.3$ to $2.5$ $\micron$ \; for 19 PNe in excellent seeing conditions and a wide slit, mimicking $\Euclid$ 's slitless spectroscopy mode but with 10 times higher spectral resolution. Additional observations of one northern PN were obtained in the $0.80$ -- $1.90$ $\micron$ range with the GMOS and GNIRS instruments at the Gemini North observatory. The collected spectra were combined into an atlas of heliocentric vacuum wavelengths with a joint statistical and systematic accuracy of 0.1 Å in the optical and 0.3 Å in the near-infrared. The wavelength atlas and the related 1D and 2D spectra are made publicly available.

</div>

<div id="div_fig1">

<img src="tmp_2303.15525/./PNmorph_linear.png" alt="Fig7" width="100%"/>

**Figure 7. -** HST images of all PNe in this paper, in linear scale (a nonlinear stretch is shown in Fig. \ref{fig:PNmorph}). DdDm-1 was taken in the F675W broad-band filter including the H$\alpha$ line. Contrary to the other panels shown here, this image of DdDm-1 suffers from spherical aberration prior to the installation of HST's corrective optics. F502N refers to a narrow-band filter including the [$\ion${O}{III}] line. These images include the emission from the central star. [$\ion${O}{III}] refers to that emission line extracted from a slitless spectrum. The spectral resolution is too low to resolve the intrinsic line width, hence these can be considered 2D line images, without the central star whose continuum is dispersed along the horizontal axis. The white circle shows the NISP-S EE80 disk with a radius of \ang{;;0.5}. The field of view is $\ang{;;2}\times\ang{;;2}$. (*fig:PNmorph_linear*)

</div>
<div id="div_fig2">

<img src="tmp_2303.15525/./PN_gaia.png" alt="Fig2" width="100%"/>

**Figure 2. -** Normalised 1st-order shape coefficients (assuming 0 indexing) of the BP and RP Gaia spectra. The logarithmic density of all 200 million Gaia sources in this space is indicated by the grey cells. Our sample of compact PNe (Table \ref{table:observations}) is shown by the red crosses if they had Gaia DR3 BP/RP spectra, and known or candidate PNe from [ and Chornay (2021)]() by light blue crosses. The latter follow the bulk of the Gaia sources, suggesting that their SEDs are continuum-dominated; a small subset shows low ${\rm RP}_{1}$ values and approach our sample of compact PNe. Compact PNe candidates with ${\rm RP}_1<-0.4$ are shown in orange, but were rejected/disqualified upon further inspection  (Sect. \ref{sec:searchGaia2}). (*fig:gaia_pn*)

</div>
<div id="div_fig3">

<img src="tmp_2303.15525/./fp_pn.png" alt="Fig4" width="100%"/>

**Figure 4. -** _Top panel_: Total system response for the NISP blue grism (shaded area, exceeding 50\% of its peak transmission between $0.93$--$1.37 $\micron; these data are updated with respect to those presented in \citetalias{scaramella2022}). The regularly spaced, grey spectrum shows the arbitrarily scaled Fabry-Perot emission lines used for the pre-flight calibration of the dispersion law.
As an example for the in-flight calibration, the purple spectrum shows the estimated NISP signal-to-noise ratio (SNR) of He2-436, extrapolated from our X-SHOOTER observations. The horizontal dashed line displays a $3.5 \sigma$ threshold to identify usable PN lines. _Bottom panel_: Same for the NISP red grism ($1.21$--$1.89 $\micron). (*fig:fp_pn*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.15525"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cmark}{\ding{51}}$
$\newcommand{\xmark}{\ding{55}}$
$\newcommand{\tder}[2]{\frac{\mathrm{d}#1}{\mathrm{d}#2}}$
$\newcommand{\pder}[2]{\frac{\partial #1}{\partial #2}}$
$\newcommand{\tctext}[1]{\begin{CJK}{UTF8}{bkai}#1\end{CJK}}$</div>



<div id="title">

# The impact of dust evolution on the dead zone outer edge in magnetized protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.15675-b31b1b.svg)](https://arxiv.org/abs/2303.15675)<mark>Appeared on: 2023-03-29</mark> -  _23 pages, 13 figures, Accepted for publication in A&A_

</div>
<div id="authors">

T. N. Delage, et al. -- incl., <mark>M. Gárate</mark>, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** The dead zone outer edge corresponds to the transition from the magnetically dead to the magnetorotational instability (MRI)-active regions in the outer protoplanetary disk mid-plane. It has been previously hypothesized to be a sweet spot for dust particles trapping. A more consistent approach to access such an idea yet remains to be developed, since the interplay between dust evolution and MRI-driven accretion over million years has been poorly understood. We provide an important step toward a better understanding of the MRI--dust coevolution in protoplanetary disks. In this pilot study, we present a proof of concept that dust evolution ultimately plays a crucial role in the MRI activity. First, we study how a fixed power-law dust size distribution with varying parameters impacts the MRI activity, especially the steady-state MRI-driven accretion, by employing and improving our previous 1+1D MRI-driven turbulence model. Second, we relax the steady-state accretion assumption in this disk accretion model, and partially couple it to a dust evolution model in order to investigate how the evolution of dust (dynamics and grain growth processes combined) and MRI-driven accretion are intertwined on million-year timescales, from a more sophisticated modeling of the gas ionization degree. Dust coagulation and settling lead to a higher gas ionization degree in the protoplanetary disk, resulting in stronger MRI-driven turbulence as well as a more compact dead zone. On the other hand, fragmentation has an opposite effect because it replenishes the disk in small dust particles which are very efficient in sweeping up free electrons and ions from the gas phase. Since the dust content of the disk decreases over million years of evolution due to radial drift, the MRI-driven turbulence overall becomes stronger and the dead zone more compact until the disk dust-gas mixture eventually behaves as a grain-free plasma. Furthermore, our results show that dust evolution alone does not lead to a complete reactivation of the dead zone. For typical T-Tauri stars, we find that the dead zone outer edge is expected to be located roughly between $10 $ au and $50 $ au during the disk lifetime for our choice of the magnetic field strength and configuration. Finally, the MRI activity evolution is expected to be crucially sensitive to the choice made for the minimum grain size of the dust distribution. The MRI activity evolution (hence the temporal evolution of the MRI-induced $\alpha$ -parameter) is controlled by dust evolution and occurs on a timescale of local dust growth, as long as there is enough dust particles in the disk to dominate the recombination process for the ionization chemistry. Once it is no longer the case, the MRI activity evolution is expected to be controlled by gas evolution and occurs on a viscous evolution timescale.

</div>

<div id="div_fig1">

<img src="tmp_2303.15675/./impact_full_dust_evolution/Steady_state_vfrag1e2/Steady_MRI_v_frag1e2_dustpy_optimal_B_field.png" alt="Fig3" width="100%"/>

**Figure 3. -** Impact of dust evolution on the MRI-driven turbulence, for Model IV. The panels show the temporal evolution of the same quantities as in Fig. \ref{fig:Vary_a_distMAX}, except for the gas accretion rate. Also, the gas surface density is now fixed to the displayed input profile in _Panel b_. We emphasize that these quantities do not describe steady-state MRI-driven accretion (unlike Fig. \ref{fig:Vary_a_distMAX}), since they are re-calculated at each dust evolution snapshot, through partial coupling between the 1D radial dust evolution model employed and our MRI-driven turbulence model. On this note, the corresponding temporal evolution of the five dust quantities used to perform such a coupling are shown in Fig. \ref{fig:Steady_MRI_v_frag1e2_key_quantities_for_MRI}. Here we note that, in _Panel f_, the dead zone outer edge coincides at $t = 0 $yr, $t = 100 $yrs and $t = 1000 $yrs.
   (*fig:Steady_MRI_v_frag1e2_dustpy_optimal_B_field*)

</div>
<div id="div_fig2">

<img src="tmp_2303.15675/./impact_dust_distribution_only/Steady-state_Vary_a_distMAX.png" alt="Fig1" width="100%"/>

**Figure 1. -** Impact on the steady-state MRI-driven accretion when varying the maximum grain size, $a_{\rm{dist,Max}}$, of the fixed power-law dust size distribution (Model I). Going from solid light-colored to dark-colored lines, $a_{\rm{dist,Max}}$ spans the range from $1   \mu$m to $1  $cm. The panels show the steady-state radial profiles of several key quantities \citep[see also][for their definition]{Delage2021}, for the model parameters $M_{\star} = 1   M_{\odot}$, $L_{\star} = 2   L_{\odot}$, $M_{\rm{disk}} = 0.05  M_\star$, $f_{\rm{dg,tot}} = 10^{-2}$, $\alpha_{\rm{hydro}} = 10^{-4}$, $a_{\rm{min}} = 0.1   \mu$m, and $p_{\rm{dist,Exp}} = -3.5$. _Panel a_: pressure-weighted vertically integrated turbulent parameter, $\bar{\alpha}$. _Panel b_: gas surface density, $\Sigma_{\rm{gas}}$. _Panel c_: mid-plane total ionization rate, $\zeta$. _Panel d_: mid-plane optimal r.m.s. magnetic field strength, $B$. _Panel e_: (constant) gas accretion rate, $\dot{M}_{\rm{acc,gas}}$. _Panel f_: mid-plane radial dead zone outer edge location, $R_{\rm{DZ}}$. For comparison, the dashed gray and black lines in _Panel a_ display the steady-state quantity $\bar{\alpha}$ obtained assuming the limiting case of a mono-disperse population of dust with size $a_{\rm{mono}} = 0.1   \mu$m and $a_{\rm{mono}} = 1  $cm (grain-free case), respectively. (*fig:Vary_a_distMAX*)

</div>
<div id="div_fig3">

<img src="tmp_2303.15675/./impact_full_dust_evolution/LBP_vfrag1e2/LBP_MRI_v_frag1e2_dustpy_optimal_B_field.png" alt="Fig5" width="100%"/>

**Figure 5. -** Same as in Fig. \ref{fig:Steady_MRI_v_frag1e2_dustpy_optimal_B_field}, except for Model V. The corresponding temporal evolution of the five dust quantities is shown in Fig. \ref{fig:LBP_MRI_v_frag1e2_key_quantities_for_MRI}. The gas surface density (_Panel b_) is fixed to the input profile corresponding to the classical self-similar solution \citep[][]{1974MNRAS.168..603L}, with a total disk gas mass $M_{\rm{disk}} = 0.05  $M$_{\star}$, and a critical radius $R_{c} = 80  $au. As a result, the gas is no longer in a steady-state MRI-driven accretion at $t = 0  $yr, unlike Model IV. Here we note that, in _Panel f_, the dead zone outer edge coincides at $t = 0 $yr and $t = 100 $yrs. (*fig:LBP_MRI_v_frag1e2_dustpy_optimal_B_field*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.15675"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

177  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
